In [1]:
import pandas as pd
import numpy as np
import os
import json
import requests
import gc
from tqdm import tqdm

pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


In [4]:
# lct = "/Users/nam-yeong/Downloads/balso_tmp"
# folders = [file for file in os.listdir(lct) if file.endswith('.csv')]
# folders.sort()
# for i in folders:
#     print(os.path.join(lct,i))
    


# df_tst = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{folders[-1]}")
# df_tst_2 = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{folders[-2]}")

df = pd.concat([df_tst,df_tst_2],axis=0)

del df_tst, df_tst_2
gc.collect()


0

In [5]:
df.head()


,Event Category,Event Action,Event Label,Event Value,Event Date,Event Datetime,Semantic Event Properties,Custom Event Properties,Total Revenue,Total Revenue (Original),Currency,Currency (Original),Product List,Product ID,Product Name,Product Price,Product Quantity,Product Currency,Product Position,Product Brand ID,Product Brand Name,Product Category ID,Product Category Name,Airbridge Device ID,Platform,User ID
0,Ad Impression (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-21,2024-04-21T23:59:59+09:00,"{""products"":[{""name"":""줄눈이오 타일줄눈, 은펄홀로그램 바닥용"",""...",{},NaN,NaN,NaN,NaN,"[{""name"":""줄눈이오 타일줄눈, 은펄홀로그램 바닥용"",""position"":0,...",NaN,"줄눈이오 타일줄눈, 은펄홀로그램 바닥용",12240.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,6a33794d-f4bb-48ef-b656-9f85a29bdecc,Android,79a63e5d-479b-4444-941f-378c266aecba
1,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-21,2024-04-21T23:59:59+09:00,"{""products"":[{""name"":""제이로사 14K반지 할로우 컷팅 우정 여자 ...",{},NaN,NaN,NaN,NaN,"[{""name"":""제이로사 14K반지 할로우 컷팅 우정 여자 실반지 레이어드 로즈골...",NaN,제이로사 14K반지 할로우 컷팅 우정 여자 실반지 레이어드 로즈골드 옐로우,52900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7d435e75-3267-4cc1-9760-6c2012ecad57,Android,18508ebc-cb65-423e-8d4e-2186fbdbf32a
2,Ad Impression (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-21,2024-04-21T23:59:59+09:00,"{""products"":[{""name"":""로탈라 홍콩(10촉)"",""position"":...",{},NaN,NaN,NaN,NaN,"[{""name"":""로탈라 홍콩(10촉)"",""position"":0,""price"":30...",NaN,로탈라 홍콩(10촉),3040.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,C8BCE4F4-1C73-49D6-8647-56EF966A5961,iOS,891a3131-03e5-4fff-88f6-51fe5ffcf7ed
3,Ad Impression (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-21,2024-04-21T23:59:59+09:00,"{""products"":[{""name"":""닥터워터 필터샤워헤드 리필 _ 대용량 12P...",{},NaN,NaN,NaN,NaN,"[{""name"":""닥터워터 필터샤워헤드 리필 _ 대용량 12P, 12개"",""posi...",NaN,"닥터워터 필터샤워헤드 리필 _ 대용량 12P, 12개",27920.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,157CF556-781F-4E89-8DBE-3A4BD80AFC82,iOS,5a1fe6b1-6a6b-440a-91a6-1014400a4be4
4,Ad Impression (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-21,2024-04-21T23:59:59+09:00,"{""products"":[{""name"":""아베른 미술 수채화 물통, 블루"",""posi...",{},NaN,NaN,NaN,NaN,"[{""name"":""아베른 미술 수채화 물통, 블루"",""position"":0,""pri...",NaN,"아베른 미술 수채화 물통, 블루",10120.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,3b1440a7-362e-4360-b386-66a4562f03d6,Android,a42f76c3-40fc-41ca-8af2-ab4aa02844f1


In [29]:
access_key = pd.read_csv('../../cp_data.csv').iloc[0,1]
secret_key = pd.read_csv('../../cp_data.csv').iloc[1,1]

import requests
import hmac
import hashlib
from time import gmtime, strftime

REQUEST_METHOD = "GET"
DOMAIN = "https://api-gateway.coupang.com"
URL = "/v2/providers/affiliate_open_api/apis/openapi/v1/reports/clicks"

# Replace with your own ACCESS_KEY, SECRET_KEY, startDate, endDate
ACCESS_KEY = access_key
SECRET_KEY = secret_key
startDate = "20240420"
endDate = "20240421"
# Assuming subId is not required, if needed add it to the query parameters

def generateHmac(method, url, secretKey, accessKey, queryString=""):
    # Correct formatting for date-time if needed
    datetimeGMT = strftime('%y%m%d', gmtime()) + 'T' + strftime('%H%M%S', gmtime()) + 'Z'
    message = datetimeGMT + method + url + queryString

    signature = hmac.new(bytes(secretKey, 'utf-8'), 
                         message.encode('utf-8'), 
                         hashlib.sha256).hexdigest()

    return "CEA algorithm=HmacSHA256, access-key={}, signed-date={}, signature={}".format(accessKey, datetimeGMT, signature)


def fetch_data(page):
    queryString = "startDate={}&endDate={}&page={}".format(startDate, endDate, page)
    fullURL = "{}{}?{}".format(DOMAIN, URL, queryString)
    authorization = generateHmac(REQUEST_METHOD, URL, SECRET_KEY, ACCESS_KEY, queryString)
    response = requests.request(method=REQUEST_METHOD, url=fullURL,
                                headers={
                                    "Authorization": authorization,
                                    "Content-Type": "application/json"
                                })
    return response.json()

def main():
    page = 1
    results = []

    while True:
        response = fetch_data(page)
        if 'data' in response:
            results.extend(response['data'])
            if len(response['data']) < 1000:
                break
            page += 1
        else:
            # Handle errors or missing 'data' key
            print(f"Error retrieving data: {response.get('message', 'No error message available')}")
            break

    print(f"Total items fetched: {len(results)}")
    return results


# Execute the main function to handle pagination
all_data = main()


Total items fetched: 32929


In [34]:
cp = pd.DataFrame(all_data)
cp.click.sum()

102990

In [37]:
len(df.query('`Event Category`.str.contains("Click")'))

98231

In [17]:
lst = cp.subParam.unique().tolist()

df.query('`User ID` in @lst')['User ID'].nunique()

770

In [22]:
df.query('`User ID`.isna()')['Airbridge Device ID'].nunique()/df['Airbridge Device ID'].nunique()

0.09953834075742836

In [26]:
df[df['User ID']]

<class 'pandas.core.frame.DataFrame'>
Index: 1323231 entries, 0 to 646733
Data columns (total 26 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Event Category             1323231 non-null  object 
 1   Event Action               1323231 non-null  object 
 2   Event Label                1323231 non-null  object 
 3   Event Value                0 non-null        float64
 4   Event Date                 1323231 non-null  object 
 5   Event Datetime             1323231 non-null  object 
 6   Semantic Event Properties  1323231 non-null  object 
 7   Custom Event Properties    1323231 non-null  object 
 8   Total Revenue              0 non-null        float64
 9   Total Revenue (Original)   0 non-null        float64
 10  Currency                   0 non-null        float64
 11  Currency (Original)        0 non-null        float64
 12  Product List               1323231 non-null  object 
 13  Product ID        